# 04.6 Quasar Tokens: Reference Directions in Semantic Space

## The Quasar Hypothesis

In astronomy, **quasars** (quasi-stellar radio sources) are extremely luminous distant objects that serve as reference points for mapping cosmic structure. They're bright enough to see from across the universe.

**Question:** Do certain tokens act as "quasars" in semantic space—bright, distant landmarks we can use as reference directions?

### Why This Matters

The 152,936 tokens in our vocabulary are scattered through 2560-dimensional space. We know they cluster in a shell roughly 50 logometers from the origin (from 04.5b/c). But within that shell:

- **Are some tokens anomalously far away?** (Deep space outliers)
- **Do special tokens (like `<|endoftext|>`) have distinctive positions?**
- **Can we use these as reference directions** for navigation and measurement?

Even with just ONE reference direction, we can:
- Measure **polarity** (alignment/projection of other tokens)
- Define semantic axes ("toward the quasar" vs "away from the quasar")
- Detect **off-manifold drift** during steering (are we heading toward empty space?)
- Characterize **special tokens** as a distinct geometric class

---

## Planned Experiments

This notebook series will explore quasar candidates and their geometric properties.

### 04.6a: Finding Quasar Candidates in Deep Space

**Goal:** Identify the most distant tokens in causal space.

**Method:**
1. Compute causal norms ||γᵢ||_M for all 152,936 tokens
2. Sort by causal norm (descending)
3. Report top 10 most distant tokens
4. Decode token IDs to see what they are
5. Check if special tokens (`<|endoftext|>`, `<|im_start|>`, etc.) appear in the list

**Hypothesis:** Special control tokens might be geometric outliers—far from the main token cluster.

**If this flunks:** Try tokens *closest* to origin instead (though these might just be rare/unused tokens).

### 04.6b: Measuring Polarity and Alignment

**Goal:** Use the most distant token as a reference direction.

**Method:**
1. Pick the #1 most distant token as our "quasar"
2. For all other tokens, compute:
   - **Alignment:** cos(θ) between token and quasar (Euclidean and causal)
   - **Projection:** component along quasar direction
   - **Orthogonal component:** what's left after removing quasar projection
3. Visualize distribution of alignments
4. Find tokens most/least aligned with the quasar

**Questions:**
- Are tokens randomly oriented relative to the quasar?
- Or is there systematic structure (clustering at certain angles)?

### 04.6c: Special Token Constellation

**Goal:** Map the geometric relationships between special tokens.

**Method:**
1. Identify all special tokens in vocabulary:
   - `<|endoftext|>`, `<|im_start|>`, `<|im_end|>`
   - `<|system|>`, `<|user|>`, `<|assistant|>`
   - Any other structural markers
2. Compute pairwise causal distances between them
3. Check their causal norms (are they outliers?)
4. Find their k-nearest neighbors in the full token set

**Questions:**
- Do special tokens form their own cluster?
- Are they scattered throughout semantic space?
- What "normal" tokens are nearest to special tokens?

### 04.6d: Distance from Quasar as a Semantic Axis

**Goal:** Use distance-from-quasar as a semantic coordinate.

**Method:**
1. For all tokens, compute d(token, quasar) in logometers
2. Bin tokens by distance (near/medium/far from quasar)
3. Sample tokens from each bin and examine them qualitatively
4. Look for semantic patterns (e.g., are content words far? Function words near?)

**Questions:**
- Does "distance from quasar" correlate with any semantic property?
- Can we use it as a meaningful coordinate axis?

### 04.6e: Steering Navigation with Quasar Reference

**Goal:** Use the quasar to detect off-manifold drift during steering.

**Method:**
1. Load a steering vector (e.g., complexity vector from 02)
2. Simulate steering: h' = h + α·v for various α
3. At each steering coefficient, measure:
   - Distance from quasar: d(h', quasar)
   - Alignment with quasar: cos(θ) between h' and quasar
   - Nearest token distance: min(d(h', γᵢ)) for all tokens
4. Plot these metrics vs steering coefficient α

**Hypothesis:** As we go off-manifold (large α), we might drift toward/away from the quasar in systematic ways.

**Application:** Could use "heading toward quasar" as a warning sign of off-manifold drift.

---

## Why Start with Distance?

We're looking for quasar *candidates* first—tokens that are geometrically exceptional. The most distant tokens are natural candidates because:

1. **They're outliers** by definition (far from typical ~50 logometer shell)
2. **They're visible from everywhere** (high norm = "bright" in our analogy)
3. **They might be structurally important** (control tokens, boundaries, null states)

From 04.1, we know the token cloud has a diameter of ~112 logometers. If typical tokens sit at radius ~50, then the most distant tokens are at **~56+ logometers**—significantly beyond the main cluster.

---

## Setup and Shared Code

The cells below load the model, tokenizer, and metric tensor. Each experiment notebook (04.6a, 04.6b, etc.) will replicate this setup in its own config cell—no cross-notebook dependencies.

If we end up reusing this code a lot, we'll extract it to `azimuth/quasars.py` following the project philosophy: "write it twice in notebooks, then it belongs in the package."

## Configuration

In [ ]:
import torch
import numpy as np
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer

# Model and paths
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
METRIC_PATH = Path("../data/vectors/causal_metric_tensor_qwen3_4b.pt")

# Device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

## Load Model, Tokenizer, and Metric Tensor

In [ ]:
# Load metric tensor
print("Loading causal metric tensor...")
metric_data = torch.load(METRIC_PATH, map_location=device, weights_only=True)
M = metric_data['M']
print(f"M shape: {M.shape}")
print()

In [ ]:
# Load model for gamma matrix
print(f"Loading {MODEL_NAME}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map=device,
)
model.eval()

# Extract gamma (unembedding matrix)
gamma = model.lm_head.weight.data.to(dtype=torch.float32, device=device)
vocab_size, hidden_dim = gamma.shape
print(f"Gamma shape: {gamma.shape}")
print(f"Vocabulary size: {vocab_size:,} tokens")
print()

In [ ]:
# Load tokenizer for decoding
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("Tokenizer loaded.")
print(f"Vocab size from tokenizer: {len(tokenizer):,}")

## Quick Verification: Find a Known Special Token

Let's verify we can decode token IDs correctly by finding `<|endoftext|>` or similar special tokens.

In [ ]:
# Look for common special tokens
special_token_names = [
    '<|endoftext|>',
    '<|im_start|>',
    '<|im_end|>',
    '<|system|>',
    '<|user|>',
    '<|assistant|>',
]

print("Looking for special tokens in vocabulary:")
print("="*60)
for token_str in special_token_names:
    try:
        # Encode the string (might return multiple tokens)
        token_ids = tokenizer.encode(token_str, add_special_tokens=False)
        if len(token_ids) == 1:
            token_id = token_ids[0]
            decoded = tokenizer.decode([token_id])
            print(f"Found: '{decoded}' (ID: {token_id})")
        else:
            print(f"'{token_str}' → {len(token_ids)} tokens (not a single token)")
    except Exception as e:
        print(f"'{token_str}' → Error: {e}")

print("\nNote: Some models use different special token syntax.")
print("We'll identify actual quasar candidates by distance in 04.6a.")

---

## Next Steps

This notebook establishes the framework. Continue to:

- **04.6a:** Find the most distant tokens in causal space (quasar candidates)
- **04.6b:** Measure polarity and alignment using the quasar as reference
- **04.6c:** Map relationships between special tokens
- **04.6d:** Use distance-from-quasar as a semantic coordinate
- **04.6e:** Apply quasar reference to steering navigation

Each experiment is self-contained and can be run independently.